<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_04_python_student_solution_Pr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# O1NAEX Exercise 04

## Setup

In [ ]:
# Skipped: !pip install rpy2

In [ ]:
# Skipped: %load_ext rpy2.ipython

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, t
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns


In [ ]:
# Recap of the Lecture in Python

# Read the data from the URL with fallback to local file
import os

rocket_path_url = "https://raw.githubusercontent.com/francji1/01NAEX/refs/heads/main/data/rocket2.txt"
rocket_path_local = "/Users/michalprusek/PycharmProjects/01NAEX/data/rocket2.txt"

try:
    rocket = pd.read_csv(rocket_path_url, sep=";")
    print("Data loaded from GitHub")
except Exception as e:
    print(f"Failed to load from GitHub: {e}")
    print("Loading from local file...")
    rocket = pd.read_csv(rocket_path_local, sep=";")
    print("Data loaded from local file")

# Renaming columns for consistency
rocket.rename(columns={'op': 'operator', 'y': 'Propellant'}, inplace=True)

# Converting columns to factors (categorical variables)
rocket['operator'] = rocket['operator'].astype('category')
rocket['batch'] = rocket['batch'].astype('category')

# Latin Square Design Plotting
sns.boxplot(x='operator', y='Propellant', data=rocket)
plt.title('Propellant by Operator')
plt.show()

sns.boxplot(x='batch', y='Propellant', data=rocket)
plt.title('Propellant by Batch')
plt.show()

sns.boxplot(x='treat', y='Propellant', data=rocket)
plt.title('Propellant by Treatment')
plt.show()

# Latin Square Design - Linear Model
rocket_lm = smf.ols('Propellant ~ operator + batch + treat', data=rocket).fit()
print("\n=== ANOVA with operator, batch, and treat = 4×4 Latin Square ===")
print(sm.stats.anova_lm(rocket_lm))

# Without considering batch as a factor
rocket_lm2 = smf.ols('Propellant ~ operator + treat', data=rocket).fit()
print("\n=== ANOVA without batch = RCBD ===")
print(sm.stats.anova_lm(rocket_lm2))


##	Problem 4.23
from the chapter 4, D. C. Montgomery DAoE - 8. edition.

An industrial engineer is investigating the effect of
four assembly methods (A, B, C, D) on the assembly time for
a color television component. Four operators are selected for
the study. Furthermore, the engineer knows that each assembly
method produces such fatigue that the time required for
the last assembly may be greater than the time required for the
first, regardless of the method. That is, a trend develops in the
required assembly time. To account for this source of variability,
the engineer uses the Latin square design shown below.
Analyze the data from this experiment (use	$\alpha = 0.05$) and draw
appropriate conclusions.



In [ ]:
# Read the data from the URL with fallback to local file
url_4_23 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_4_23.txt"
local_4_23 = "/Users/michalprusek/PycharmProjects/01NAEX/data/Problem_4_23.txt"

try:
    df_4_23 = pd.read_csv(url_4_23, sep=";")
    print("Data loaded from GitHub")
except Exception as e:
    print(f"Failed to load from GitHub: {e}")
    print("Loading from local file...")
    df_4_23 = pd.read_csv(local_4_23, sep=";")
    print("Data loaded from local file")

# Display the first few rows of the dataframe
print("\n=== Problem 4.23 Data ===")
print(df_4_23.head())
print(f"\nData shape: {df_4_23.shape}")
print(f"\nColumns: {df_4_23.columns.tolist()}")

### Problem 4.23 - Analysis

**Latin Square Design:** 4 assembly methods (A, B, C, D) × 4 operators × 4 orders

This is a Latin Square design where:
- **Treatments**: Assembly methods (A, B, C, D)
- **Row blocking factor**: Order (1-4, accounts for fatigue effect)
- **Column blocking factor**: Operator (1-4)

We will perform ANOVA with α = 0.05 and draw conclusions.

In [ ]:
# Problem 4.23 - Latin Square Analysis
from scipy import stats

# Convert columns to categorical
df_4_23['Operator'] = df_4_23['Operator'].astype('category')
df_4_23['Order'] = df_4_23['Order'].astype('category')
df_4_23['Method'] = df_4_23['Method'].astype('category')

# Summary statistics
print("=== Summary Statistics by Assembly Method ===")
print(df_4_23.groupby('Method', observed=True)['Time'].describe())

print("\n=== Summary Statistics by Operator ===")
print(df_4_23.groupby('Operator', observed=True)['Time'].describe())

print("\n=== Summary Statistics by Order ===")
print(df_4_23.groupby('Order', observed=True)['Time'].describe())

# Check the design structure (Latin square)
print("\n=== Latin Square Structure ===")
print(pd.crosstab(df_4_23['Operator'], df_4_23['Order'], values=df_4_23['Method'], aggfunc=lambda x: x.iloc[0]))

# Visualizations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.boxplot(x='Method', y='Time', data=df_4_23, ax=axes[0])
axes[0].set_title('Assembly Time by Method')
axes[0].set_ylabel('Time')

sns.boxplot(x='Operator', y='Time', data=df_4_23, ax=axes[1])
axes[1].set_title('Assembly Time by Operator')
axes[1].set_ylabel('Time')

sns.boxplot(x='Order', y='Time', data=df_4_23, ax=axes[2])
axes[2].set_title('Assembly Time by Order (Fatigue Effect)')
axes[2].set_ylabel('Time')

plt.tight_layout()
plt.show()

# Latin Square ANOVA Model
model_4_23 = smf.ols('Time ~ Operator + Order + Method', data=df_4_23).fit()
anova_4_23 = sm.stats.anova_lm(model_4_23, typ=2)

print("\n=== ANOVA Table (Latin Square Design) ===")
print(anova_4_23)
print(f"\nModel R-squared: {model_4_23.rsquared:.4f}")

# Test for significance at α = 0.05
alpha = 0.05
print(f"\n=== Hypothesis Testing (α = {alpha}) ===")
print(f"Operators: F = {anova_4_23.loc['Operator', 'F']:.4f}, p-value = {anova_4_23.loc['Operator', 'PR(>F)']:.4f}")
if anova_4_23.loc['Operator', 'PR(>F)'] < alpha:
    print("  → Operators have a SIGNIFICANT effect on assembly time")
else:
    print("  → Operators do NOT have a significant effect on assembly time")

print(f"\nOrder: F = {anova_4_23.loc['Order', 'F']:.4f}, p-value = {anova_4_23.loc['Order', 'PR(>F)']:.4f}")
if anova_4_23.loc['Order', 'PR(>F)'] < alpha:
    print("  → Order (fatigue) has a SIGNIFICANT effect on assembly time")
else:
    print("  → Order (fatigue) does NOT have a significant effect on assembly time")

print(f"\nAssembly Method: F = {anova_4_23.loc['Method', 'F']:.4f}, p-value = {anova_4_23.loc['Method', 'PR(>F)']:.4f}")
if anova_4_23.loc['Method', 'PR(>F)'] < alpha:
    print("  → Assembly methods have a SIGNIFICANT effect on assembly time")
else:
    print("  → Assembly methods do NOT have a significant effect on assembly time")

# Pairwise comparisons if methods are significant
if anova_4_23.loc['Method', 'PR(>F)'] < alpha:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    print("\n=== Tukey HSD Post-hoc Test ===")
    tukey_4_23 = pairwise_tukeyhsd(endog=df_4_23['Time'],
                                    groups=df_4_23['Method'],
                                    alpha=alpha)
    print(tukey_4_23)

# Model diagnostics with STUDENTIZED RESIDUALS
print("\n=== Model Diagnostics ===")

# Get influence measures for studentized residuals
influence_4_23 = model_4_23.get_influence()
studentized_residuals_4_23 = influence_4_23.resid_studentized_external
fitted_4_23 = model_4_23.fittedvalues

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Studentized Residuals vs Fitted
axes[0].scatter(fitted_4_23, studentized_residuals_4_23)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].axhline(y=2, color='orange', linestyle='--', alpha=0.5)
axes[0].axhline(y=-2, color='orange', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Fitted values')
axes[0].set_ylabel('Studentized Residuals')
axes[0].set_title('Studentized Residuals vs Fitted')

# Q-Q plot of studentized residuals
stats.probplot(studentized_residuals_4_23, dist="norm", plot=axes[1])
axes[1].set_title('Normal Q-Q Plot (Studentized Residuals)')

# Histogram of studentized residuals
axes[2].hist(studentized_residuals_4_23, bins=10, edgecolor='black')
axes[2].axvline(x=-2, color='orange', linestyle='--', alpha=0.5)
axes[2].axvline(x=2, color='orange', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Studentized Residuals')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Histogram of Studentized Residuals')

plt.tight_layout()
plt.show()

# Check for outliers (|studentized residual| > 2)
outliers_4_23 = np.abs(studentized_residuals_4_23) > 2
if outliers_4_23.any():
    print(f"\n⚠ Warning: {outliers_4_23.sum()} potential outlier(s) detected (|studentized residual| > 2)")
    print(f"Observation indices: {np.where(outliers_4_23)[0]}")
else:
    print("\n✓ No significant outliers detected (all |studentized residuals| ≤ 2)")

### Problem 4.23 - Results Interpretation

#### ANOVA Summary

**Latin Square Design** with two-way blocking (operator × order)

| Factor | F-value | p-value | Significance (α=0.05) | Conclusion |
|--------|---------|---------|----------------------|------------|
| **Operator** | 9.81 | 0.0099 | ✅ Yes | Significant effect |
| **Order (fatigue)** | 3.52 | 0.0885 | ❌ No | Not significant |
| **Method (assembly method)** | 13.81 | 0.0042 | ✅ Yes | Significant effect |

**R² = 0.9524** - Model explains 95.24% of data variability

**Latin Square Structure:**
- 4 assembly methods (A, B, C, D) tested by 4 operators in 4 orders
- Each operator uses each method exactly once
- Each order position contains each method exactly once
- Blocks for both operator and order (fatigue) effects

#### Main Conclusions:

1. **Assembly methods (A, B, C, D) have a statistically significant effect** on assembly time (p = 0.0042)
   - Different methods lead to different assembly times
   - This effect is the main finding of the experiment
   
2. **Operators have a statistically significant effect** (p = 0.0099)
   - Different operators work at different speeds
   - Blocking by operator was justified and helped increase precision
   
3. **Assembly order (fatigue) does NOT have a statistically significant effect** (p = 0.0885 > 0.05)
   - The assumption that fatigue affects time was not confirmed
   - Possible reasons: short experiment duration, learning effect compensates fatigue

#### Tukey HSD Post-hoc Test:

The Tukey HSD test will identify which specific assembly methods differ significantly from each other. Since the overall ANOVA shows methods are significant (p = 0.0042), we examine pairwise comparisons to determine which methods perform differently.

#### Model Diagnostics:

- ⚠️ **2 potential outliers** detected (|studentized residual| > 2)
- Q-Q plot shows minor deviations from normality
- Recommendation: Check measurements at outliers, but model is generally reliable

#### Practical Recommendations:

1. **Focus on selecting the optimal assembly method** (main significant factor)
2. Account for individual differences between operators when planning capacity
3. Fatigue effect is not a problem within 4 assemblies
4. Consider Tukey test results to identify best and worst methods

## Problem  4.40
from the chapter 4, D. C. Montgomery DAoE - 8. edition.



An engineer is studying the mileage performance
characteristics of five types of gasoline additives. In the road
test he wishes to use cars as blocks; however, because of a time constraint, he must use an incomplete block design. He
runs the balanced design with the five blocks that follow.
Analyze the data from this experiment (use $\alpha	 = 0.05$) and
draw conclusions.


In [ ]:
# Read the data from the URL with fallback to local file
url_4_40 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_4_40.txt"
local_4_40 = "/Users/michalprusek/PycharmProjects/01NAEX/data/Problem_4_40.txt"

try:
    df_4_40 = pd.read_csv(url_4_40, sep=";")
    print("Data loaded from GitHub")
except Exception as e:
    print(f"Failed to load from GitHub: {e}")
    print("Loading from local file...")
    df_4_40 = pd.read_csv(local_4_40, sep=";")
    print("Data loaded from local file")

# Display the first few rows of the dataframe
print("\n=== Problem 4.40 Data ===")
print(df_4_40.head())
print(f"\nData shape: {df_4_40.shape}")
print(f"\nColumns: {df_4_40.columns.tolist()}")

In [ ]:
# Problem 4.40 - BIBD Analysis
from scipy import stats

# Convert columns to categorical
df_4_40['Car'] = df_4_40['Car'].astype('category')
df_4_40['Additive'] = df_4_40['Additive'].astype('category')

# Summary statistics
print("=== Summary Statistics by Gasoline Additive ===")
print(df_4_40.groupby('Additive', observed=True)['Mileage'].describe())

print("\n=== Summary Statistics by Car (Block) ===")
print(df_4_40.groupby('Car', observed=True)['Mileage'].describe())

# Check the design structure (incidence matrix)
print("\n=== BIBD Structure (treatments in each block) ===")
print(pd.crosstab(df_4_40['Car'], df_4_40['Additive']))

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.boxplot(x='Additive', y='Mileage', data=df_4_40, ax=axes[0])
axes[0].set_title('Mileage by Gasoline Additive')
axes[0].set_xlabel('Additive')
axes[0].set_ylabel('Mileage')

sns.boxplot(x='Car', y='Mileage', data=df_4_40, ax=axes[1])
axes[1].set_title('Mileage by Car (Block)')
axes[1].set_xlabel('Car')
axes[1].set_ylabel('Mileage')

plt.tight_layout()
plt.show()

# BIBD ANOVA Model
# For incomplete block designs, we use the standard ANOVA approach
model_4_40 = smf.ols('Mileage ~ Car + Additive', data=df_4_40).fit()
anova_4_40 = sm.stats.anova_lm(model_4_40, typ=2)

print("\n=== ANOVA Table (BIBD) ===")
print(anova_4_40)
print(f"\nModel R-squared: {model_4_40.rsquared:.4f}")

# Test for significance at α = 0.05
alpha = 0.05
print(f"\n=== Hypothesis Testing (α = {alpha}) ===")
print(f"Car (Block): F = {anova_4_40.loc['Car', 'F']:.4f}, p-value = {anova_4_40.loc['Car', 'PR(>F)']:.4f}")
if anova_4_40.loc['Car', 'PR(>F)'] < alpha:
    print("  → Cars (blocks) have a SIGNIFICANT effect on mileage")
else:
    print("  → Cars (blocks) do NOT have a significant effect on mileage")

print(f"\nGasoline Additive: F = {anova_4_40.loc['Additive', 'F']:.4f}, p-value = {anova_4_40.loc['Additive', 'PR(>F)']:.4f}")
if anova_4_40.loc['Additive', 'PR(>F)'] < alpha:
    print("  → Gasoline additives have a SIGNIFICANT effect on mileage")
else:
    print("  → Gasoline additives do NOT have a significant effect on mileage")

# Pairwise comparisons if additives are significant
if anova_4_40.loc['Additive', 'PR(>F)'] < alpha:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    print("\n=== Tukey HSD Post-hoc Test ===")
    tukey_4_40 = pairwise_tukeyhsd(endog=df_4_40['Mileage'],
                                    groups=df_4_40['Additive'],
                                    alpha=alpha)
    print(tukey_4_40)

# Model diagnostics with STUDENTIZED RESIDUALS
print("\n=== Model Diagnostics ===" )

# Get influence measures for studentized residuals
influence_4_40 = model_4_40.get_influence()
studentized_residuals_4_40 = influence_4_40.resid_studentized_external
fitted_4_40 = model_4_40.fittedvalues

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Studentized Residuals vs Fitted
axes[0].scatter(fitted_4_40, studentized_residuals_4_40)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].axhline(y=2, color='orange', linestyle='--', alpha=0.5)
axes[0].axhline(y=-2, color='orange', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Fitted values')
axes[0].set_ylabel('Studentized Residuals')
axes[0].set_title('Studentized Residuals vs Fitted')

# Q-Q plot of studentized residuals
stats.probplot(studentized_residuals_4_40, dist="norm", plot=axes[1])
axes[1].set_title('Normal Q-Q Plot (Studentized Residuals)')

# Histogram of studentized residuals
axes[2].hist(studentized_residuals_4_40, bins=10, edgecolor='black')
axes[2].axvline(x=-2, color='orange', linestyle='--', alpha=0.5)
axes[2].axvline(x=2, color='orange', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Studentized Residuals')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Histogram of Studentized Residuals')

plt.tight_layout()
plt.show()

# Check for outliers (|studentized residual| > 2)
outliers_4_40 = np.abs(studentized_residuals_4_40) > 2
if outliers_4_40.any():
    print(f"\n⚠ Warning: {outliers_4_40.sum()} potential outlier(s) detected (|studentized residual| > 2)")
    print(f"Observation indices: {np.where(outliers_4_40)[0]}")
else:
    print("\n✓ No significant outliers detected (all |studentized residuals| ≤ 2)")

### Problem 4.40 - Results Interpretation

---

## 1️⃣ ANOVA Summary

**Balanced Incomplete Block Design (BIBD)** - 5 additives × 5 cars (incomplete blocks)

| Factor | F-value | p-value | Significance (α=0.05) | Conclusion |
|--------|---------|---------|----------------------|------------|
| **Car (block)** | 9.67 | 0.0013 | ✅ Yes | Significant effect |
| **Additive** | 9.81 | 0.0012 | ✅ Yes | Significant effect |

**R² = 0.8698** - Model explains 86.98% of data variability

---

## 2️⃣ Experimental Design

**BIBD Structure:**
- 5 gasoline additives tested in 5 cars (blocks)
- Each car tested **4 out of 5** additives (incomplete design)
- Each pair of additives appears together the same number of times → **balanced design**
- Total: 20 observations (5 cars × 4 additives/car)

**Why BIBD?**
- Time constraint prevents testing all additives in all cars
- Balanced structure ensures fair comparison between additives
- Blocking by car eliminates vehicle-to-vehicle variability

---

## 3️⃣ Main Findings

### ✅ Gasoline Additives: SIGNIFICANT effect (p = 0.0012)

**Ranking by average mileage:**

| Rank | Additive | Avg Mileage (mpg) | Performance |
|------|----------|-------------------|-------------|
| 🥇 1st | Additive 1 | 14.00 | Best |
| 🥈 2nd | Additive 2 | 12.75 | Good |
| 🥉 3rd | Additive 4 | 11.75 | Average |
| 4th | Additive 3 | 11.50 | Below average |
| 5th | Additive 5 | 10.25 | Worst |

**Range:** 3.75 mpg difference between best and worst (14.00 - 10.25)

### ✅ Cars (Blocks): SIGNIFICANT effect (p = 0.0013)

- Different cars have **inherently different fuel consumption**
- This justifies blocking by car → improves precision
- Blocking successfully eliminated vehicle-to-vehicle variability

---

## 4️⃣ Tukey HSD Post-hoc Test

### ⚠️ Apparent Paradox:

- **Overall ANOVA:** Additives ARE significant (p = 0.0012) ✅
- **Pairwise comparisons:** NO pairs significantly different (all p > 0.05) ❌

### Why this paradox?

1. **Small sample size:** Only n=4 observations per additive
2. **Conservative test:** Tukey HSD is more conservative than F-test
3. **Multiple comparisons:** Tukey adjusts for 10 pairwise comparisons (5 choose 2)
4. **Large within-group variance:** Variability reduces statistical power

### Interpretation:

- **F-test** detects overall pattern across all groups (significant)
- **Tukey test** cannot identify specific pairs due to small sample size
- **Practical vs Statistical significance:** 3.75 mpg difference MAY be practically important, but not statistically significant at α=0.05

---

## 5️⃣ Model Diagnostics

### Assumptions Check:

✅ **Normality:** Q-Q plot shows good normality (except outliers)  
⚠️ **Outliers:** 2 potential outliers detected (|studentized residual| > 2)  
✅ **Model fit:** R² = 0.87 (good fit)

### Recommendation:
- From my POV, Model is generally **reliable**
- I would consider removing 2 potential outliers if measurement errors suspected

---

## 6️⃣ Practical Recommendations

### For Immediate Application:

1. **Best choice:** Use **Additive 1** (14.00 mpg average)
   - Consistently highest mileage performance
   - 3.75 mpg better than worst additive (Additive 5)

2. **Worst choice:** Avoid **Additive 5** (10.25 mpg average)
   - Lowest mileage performance
   - 3.75 mpg worse than best additive

# Problem  4.42
from the chapter 4, D. C. Montgomery DAoE - 8. edition.\\[3mm]


Seven different hardwood concentrations are being studied to determine their effect on the strength of the paper produced. However, the pilot plant can only produce three	runs each day. As days may differ, the analyst uses the balanced incomplete block design that follows. Analyze the data from this experiment (use $\alpha = 0.05$) and draw conclusions.

Try to run, in addition to ANOVA with BIBD, the linear model with concentration as a quantitative response too (on condition there is no day effect).



In [ ]:
# Read the data from the URL with fallback to local file
url_4_42 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_4_42.txt"
local_4_42 = "/Users/michalprusek/PycharmProjects/01NAEX/data/Problem_4_42.txt"

try:
    df_4_42 = pd.read_csv(url_4_42, sep=";")
    print("Data loaded from GitHub")
except Exception as e:
    print(f"Failed to load from GitHub: {e}")
    print("Loading from local file...")
    df_4_42 = pd.read_csv(local_4_42, sep=";")
    print("Data loaded from local file")

# Display the first few rows of the dataframe
print("\n=== Problem 4.42 Data ===")
print(df_4_42.head())
print(f"\nData shape: {df_4_42.shape}")
print(f"\nColumns: {df_4_42.columns.tolist()}")

### Problem 4.42 - Analysis

**Balanced Incomplete Block Design (BIBD):** 7 hardwood concentrations × 7 days (blocks)

This is a BIBD where:
- **Treatments**: 7 hardwood concentrations
- **Blocks**: 7 days (each day produces only 3 runs - incomplete blocks)
- Not all concentrations are tested each day (incomplete design)
- The design is balanced: each pair of treatments appears together the same number of times

We will:
1. Perform ANOVA with α = 0.05 for the BIBD
2. If the day effect is not significant, also fit a linear model with concentration as a quantitative variable

In [ ]:
# Problem 4.42 - BIBD Analysis
from scipy import stats

# Convert columns to categorical for ANOVA
df_4_42['Days'] = df_4_42['Days'].astype('category')
df_4_42['Concentration'] = df_4_42['Concentration'].astype('category')

# Keep a numeric version for linear regression
df_4_42['concentration_numeric'] = pd.to_numeric(df_4_42['Concentration'].astype(str))

# Summary statistics
print("=== Summary Statistics by Hardwood Concentration ===")
print(df_4_42.groupby('Concentration', observed=True)['Strength'].describe())

print("\n=== Summary Statistics by Day (Block) ===")
print(df_4_42.groupby('Days', observed=True)['Strength'].describe())

# Check the design structure (incidence matrix)
print("\n=== BIBD Structure (concentrations tested each day) ===")
print(pd.crosstab(df_4_42['Days'], df_4_42['Concentration']))

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.boxplot(x='Concentration', y='Strength', data=df_4_42, ax=axes[0])
axes[0].set_title('Paper Strength by Hardwood Concentration')
axes[0].set_xlabel('Concentration')
axes[0].set_ylabel('Strength')

sns.boxplot(x='Days', y='Strength', data=df_4_42, ax=axes[1])
axes[1].set_title('Paper Strength by Day (Block)')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Strength')

plt.tight_layout()
plt.show()

# BIBD ANOVA Model
model_4_42 = smf.ols('Strength ~ Days + Concentration', data=df_4_42).fit()
anova_4_42 = sm.stats.anova_lm(model_4_42, typ=2)

print("\n=== ANOVA Table (BIBD) ===")
print(anova_4_42)
print(f"\nModel R-squared: {model_4_42.rsquared:.4f}")

# Test for significance at α = 0.05
alpha = 0.05
print(f"\n=== Hypothesis Testing (α = {alpha}) ===")
print(f"Day (Block): F = {anova_4_42.loc['Days', 'F']:.4f}, p-value = {anova_4_42.loc['Days', 'PR(>F)']:.4f}")
day_significant = anova_4_42.loc['Days', 'PR(>F)'] < alpha
if day_significant:
    print("  → Days (blocks) have a SIGNIFICANT effect on strength")
else:
    print("  → Days (blocks) do NOT have a significant effect on strength")

print(f"\nHardwood Concentration: F = {anova_4_42.loc['Concentration', 'F']:.4f}, p-value = {anova_4_42.loc['Concentration', 'PR(>F)']:.4f}")
if anova_4_42.loc['Concentration', 'PR(>F)'] < alpha:
    print("  → Hardwood concentrations have a SIGNIFICANT effect on strength")
else:
    print("  → Hardwood concentrations do NOT have a significant effect on strength")

# Pairwise comparisons if concentrations are significant
if anova_4_42.loc['Concentration', 'PR(>F)'] < alpha:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    print("\n=== Tukey HSD Post-hoc Test ===")
    tukey_4_42 = pairwise_tukeyhsd(endog=df_4_42['Strength'],
                                    groups=df_4_42['Concentration'],
                                    alpha=alpha)
    print(tukey_4_42)

# Model diagnostics with STUDENTIZED RESIDUALS
print("\n=== Model Diagnostics (BIBD) ===" )

# Get influence measures for studentized residuals
influence_4_42 = model_4_42.get_influence()
studentized_residuals_4_42 = influence_4_42.resid_studentized_external
fitted_4_42 = model_4_42.fittedvalues

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Studentized Residuals vs Fitted
axes[0].scatter(fitted_4_42, studentized_residuals_4_42)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].axhline(y=2, color='orange', linestyle='--', alpha=0.5)
axes[0].axhline(y=-2, color='orange', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Fitted values')
axes[0].set_ylabel('Studentized Residuals')
axes[0].set_title('Studentized Residuals vs Fitted (BIBD)')

# Q-Q plot of studentized residuals
stats.probplot(studentized_residuals_4_42, dist="norm", plot=axes[1])
axes[1].set_title('Normal Q-Q Plot (Studentized Residuals)')

# Histogram of studentized residuals
axes[2].hist(studentized_residuals_4_42, bins=10, edgecolor='black')
axes[2].axvline(x=-2, color='orange', linestyle='--', alpha=0.5)
axes[2].axvline(x=2, color='orange', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Studentized Residuals')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Histogram of Studentized Residuals')

plt.tight_layout()
plt.show()

# Check for outliers (|studentized residual| > 2)
outliers_4_42 = np.abs(studentized_residuals_4_42) > 2
if outliers_4_42.any():
    print(f"\n⚠ Warning: {outliers_4_42.sum()} potential outlier(s) detected (|studentized residual| > 2)")
    print(f"Observation indices: {np.where(outliers_4_42)[0]}")
else:
    print("\n✓ No significant outliers detected (all |studentized residuals| ≤ 2)")

# Linear regression with concentration as quantitative variable
print("\n" + "="*70)
print("=== LINEAR REGRESSION WITH CONCENTRATION AS QUANTITATIVE VARIABLE ===")
print("="*70)

# ALWAYS use day blocking for better precision
# Even though days are not significant (p=0.0701), blocking reduces residual variance
print("\n⚠️ NOTE: While days are not statistically significant (p=0.0701),")
print("we include them as a blocking factor to improve precision of the")
print("concentration effect estimate.\n")

# Linear regression with day blocking
model_4_42_linear = smf.ols('Strength ~ Days + concentration_numeric', data=df_4_42).fit()
print("=== Linear Model (with day blocking): strength ~ days + concentration ===")
print(model_4_42_linear.summary())

# Test significance of concentration slope
conc_pvalue = model_4_42_linear.pvalues['concentration_numeric']
conc_coef = model_4_42_linear.params['concentration_numeric']
conc_stderr = model_4_42_linear.bse['concentration_numeric']

print("\n" + "="*70)
print("CONCENTRATION EFFECT SUMMARY")
print("="*70)
print(f"Slope coefficient: {conc_coef:.4f}")
print(f"Standard error: {conc_stderr:.4f}")
print(f"p-value: {conc_pvalue:.4f}")
print(f"95% CI: [{model_4_42_linear.conf_int().loc['concentration_numeric', 0]:.4f}, "
      f"{model_4_42_linear.conf_int().loc['concentration_numeric', 1]:.4f}]")

if conc_pvalue < 0.05:
    print(f"\n✅ Concentration has a SIGNIFICANT linear effect (p = {conc_pvalue:.4f})")
    print(f"   Interpretation: Each 1% increase in concentration → +{conc_coef:.4f} units of strength")
else:
    print(f"\n⚠️ Concentration does NOT have a significant linear effect (p = {conc_pvalue:.4f})")
    print(f"   This suggests the relationship may be NON-LINEAR!")

# Scatter plot with regression line (adjusted for day effect)
plt.figure(figsize=(10, 6))

# Plot points colored by day
colors = plt.cm.tab10(np.linspace(0, 1, len(df_4_42['Days'].unique())))
for idx, day_val in enumerate(sorted(df_4_42['Days'].unique())):
    day_data = df_4_42[df_4_42['Days'] == day_val]
    plt.scatter(day_data['concentration_numeric'], day_data['Strength'],
               alpha=0.7, s=100, color=colors[idx], label=f'Day {day_val}', edgecolors='black')

# Add overall trend line (marginal effect of concentration)
x_pred = np.linspace(df_4_42['concentration_numeric'].min(),
                     df_4_42['concentration_numeric'].max(), 100)
# Use mean day effect (average across all days)
y_pred = (model_4_42_linear.params['Intercept'] +
          model_4_42_linear.params['concentration_numeric'] * x_pred)
plt.plot(x_pred, y_pred, 'r--', linewidth=3, label=f'Linear trend (slope={conc_coef:.2f})', alpha=0.8)

# Add confidence interval
from scipy import stats as sp_stats
predict_df = pd.DataFrame({'concentration_numeric': x_pred})
# Add dummy day variable (use first day for prediction)
predict_df['Days'] = df_4_42['Days'].iloc[0]
predictions = model_4_42_linear.get_prediction(predict_df)
pred_summary = predictions.summary_frame(alpha=0.05)
plt.fill_between(x_pred, pred_summary['mean_ci_lower'], pred_summary['mean_ci_upper'],
                 color='red', alpha=0.2, label='95% Confidence interval')

plt.xlabel('Hardwood Concentration (%)', fontsize=12)
plt.ylabel('Paper Strength', fontsize=12)
plt.title('Linear Regression: Paper Strength vs Hardwood Concentration\n(with Day blocking)', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Also show mean by concentration for comparison
print("\n" + "="*70)
print("COMPARISON: Group means vs Linear prediction")
print("="*70)
conc_means = df_4_42.groupby('concentration_numeric')['Strength'].mean().sort_index()
print("\nObserved means by concentration:")
for conc, mean_val in conc_means.items():
    # Predict using model (with average day effect)
    pred_val = model_4_42_linear.params['Intercept'] + model_4_42_linear.params['concentration_numeric'] * conc
    diff = mean_val - pred_val
    print(f"  Conc {conc:2.0f}%: Observed = {mean_val:6.2f}, Linear prediction = {pred_val:6.2f}, Diff = {diff:+6.2f}")

print("\n⚠️ IMPORTANT: Look for non-monotonic pattern in the differences!")
print("   If differences show systematic pattern, linear model is inadequate.")


# Day 4 was a really good day :)

In [ ]:
# Problem 4.42 - Advanced Variance Stabilization
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("=== VARIANCE STABILIZATION: MULTIPLE APPROACHES ===")
print("=" * 80)

print("\n⚠️ PROBLEM IDENTIFIED: Heteroscedasticity in residuals")
print("   - Large variance at low fitted values")
print("   - Variance decreases with higher fitted values")
print("   - This violates the constant variance assumption of ANOVA\n")

# ============================================
# APPROACH 1: Box-Cox Transformation
# ============================================
print("=" * 80)
print("APPROACH 1: Box-Cox Transformation")
print("=" * 80)

strength_transformed, lambda_optimal = boxcox(df_4_42['Strength'])
print(f"\nOptimal λ (lambda) = {lambda_optimal:.4f}")

df_4_42['Strength_boxcox'] = strength_transformed
model_boxcox = smf.ols('Strength_boxcox ~ Days + Concentration', data=df_4_42).fit()
influence_boxcox = model_boxcox.get_influence()
resid_boxcox = influence_boxcox.resid_studentized_external
fitted_boxcox = model_boxcox.fittedvalues

print(f"R² = {model_boxcox.rsquared:.4f}")

# ============================================
# APPROACH 2: Log Transformation
# ============================================
print("\n" + "=" * 80)
print("APPROACH 2: Natural Log Transformation (λ = 0)")
print("=" * 80)

df_4_42['Strength_log'] = np.log(df_4_42['Strength'])
model_log = smf.ols('Strength_log ~ Days + Concentration', data=df_4_42).fit()
influence_log = model_log.get_influence()
resid_log = influence_log.resid_studentized_external
fitted_log = model_log.fittedvalues

print(f"\nR² = {model_log.rsquared:.4f}")

# ============================================
# APPROACH 3: Square Root Transformation
# ============================================
print("\n" + "=" * 80)
print("APPROACH 3: Square Root Transformation (λ = 0.5)")
print("=" * 80)

df_4_42['Strength_sqrt'] = np.sqrt(df_4_42['Strength'])
model_sqrt = smf.ols('Strength_sqrt ~ Days + Concentration', data=df_4_42).fit()
influence_sqrt = model_sqrt.get_influence()
resid_sqrt = influence_sqrt.resid_studentized_external
fitted_sqrt = model_sqrt.fittedvalues

print(f"\nR² = {model_sqrt.rsquared:.4f}")

# ============================================
# APPROACH 4: Weighted Least Squares (WLS)
# ============================================
print("\n" + "=" * 80)
print("APPROACH 4: Weighted Least Squares (WLS)")
print("=" * 80)
print("\nWeights based on inverse variance per concentration group")

# Calculate variance for each concentration
from statsmodels.formula.api import wls

# First fit OLS to get residuals
model_ols_temp = smf.ols('Strength ~ Days + Concentration', data=df_4_42).fit()
df_4_42['resid_temp'] = model_ols_temp.resid

# Calculate squared residuals for each concentration
df_4_42['resid_sq'] = df_4_42['resid_temp']**2

# Get variance estimate for each concentration (using numeric concentration)
df_4_42['Concentration_numeric_for_wls'] = pd.to_numeric(df_4_42['Concentration'].astype(str))
variance_by_conc = df_4_42.groupby('Concentration_numeric_for_wls')['resid_sq'].mean()
print("\nVariance by concentration:")
print(variance_by_conc)

# Map variance to each observation and convert to float
df_4_42['variance_est'] = df_4_42['Concentration_numeric_for_wls'].map(variance_by_conc).astype(float)

# Weights are inverse of variance
df_4_42['weights'] = 1.0 / df_4_42['variance_est']

# Fit WLS model
model_wls = wls('Strength ~ Days + Concentration', data=df_4_42, weights=df_4_42['weights']).fit()

# For WLS, use Pearson residuals (already weighted and standardized)
resid_wls = model_wls.resid_pearson
fitted_wls = model_wls.fittedvalues

print(f"\nR² = {model_wls.rsquared:.4f}")

# ============================================

# ============================================
# APPROACH 5: Improved WLS (Fitted-Value Based)
# ============================================
print("\n" + "=" * 80)
print("APPROACH 5: Improved Weighted Least Squares")
print("=" * 80)
print("\nWeights based on inverse of fitted values (addresses funnel pattern)\n")

# Model variance as function of fitted values
# Fit OLS first to get fitted values
model_ols_for_weights = smf.ols('Strength ~ Days + Concentration', data=df_4_42).fit()
fitted_ols = model_ols_for_weights.fittedvalues

# Variance increases as fitted decreases (funnel pattern)
# Weight by inverse of fitted value (or squared)
df_4_42['weights_fitted'] = 1.0 / fitted_ols

# Fit improved WLS
model_wls_improved = wls('Strength ~ Days + Concentration',
                         data=df_4_42,
                         weights=df_4_42['weights_fitted']).fit()

resid_wls_improved = model_wls_improved.resid_pearson
fitted_wls_improved = model_wls_improved.fittedvalues

print(f"R² = {model_wls_improved.rsquared:.4f}")

# ============================================
# VISUAL COMPARISON OF ALL APPROACHES
# ============================================
fig, axes = plt.subplots(6, 3, figsize=(18, 24))
fig.suptitle('Variance Stabilization: Comparison of Different Approaches', fontsize=16, fontweight='bold')

approaches = [
    ('ORIGINAL', fitted_4_42, studentized_residuals_4_42, 'blue'),
    (f'BOX-COX (λ={lambda_optimal:.3f})', fitted_boxcox, resid_boxcox, 'green'),
    ('LOG Transform', fitted_log, resid_log, 'purple'),
    ('SQRT Transform', fitted_sqrt, resid_sqrt, 'orange'),
    ('WLS (by Conc)', fitted_wls, resid_wls, 'brown'),
    ('WLS (by Fitted)', fitted_wls_improved, resid_wls_improved, 'red')
]

for i, (name, fitted, resid, color) in enumerate(approaches):
    # Residuals vs Fitted
    axes[i, 0].scatter(fitted, resid, alpha=0.6, color=color)
    axes[i, 0].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[i, 0].axhline(y=2, color='orange', linestyle='--', alpha=0.5)
    axes[i, 0].axhline(y=-2, color='orange', linestyle='--', alpha=0.5)
    axes[i, 0].set_xlabel('Fitted values')
    axes[i, 0].set_ylabel('Standardized Residuals')
    axes[i, 0].set_title(f'{name}: Residuals vs Fitted', fontsize=10, fontweight='bold')
    axes[i, 0].grid(True, alpha=0.3)

    # Calculate spread at low vs high fitted values
    median_fitted = np.median(fitted)
    low_var = resid[fitted < median_fitted].var()
    high_var = resid[fitted >= median_fitted].var()
    var_ratio = low_var / high_var if high_var > 0 else np.inf
    axes[i, 0].text(0.02, 0.98, f'Var ratio (low/high): {var_ratio:.2f}',
                   transform=axes[i, 0].transAxes, fontsize=8,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # Q-Q plot
    stats.probplot(resid, dist="norm", plot=axes[i, 1])
    axes[i, 1].get_lines()[0].set_color(color)
    axes[i, 1].get_lines()[1].set_color('black')
    axes[i, 1].set_title(f'{name}: Q-Q Plot', fontsize=10, fontweight='bold')
    axes[i, 1].grid(True, alpha=0.3)

    # Histogram
    axes[i, 2].hist(resid, bins=10, edgecolor='black', alpha=0.7, color=color)
    axes[i, 2].axvline(x=-2, color='orange', linestyle='--', alpha=0.5)
    axes[i, 2].axvline(x=2, color='orange', linestyle='--', alpha=0.5)
    axes[i, 2].set_xlabel('Standardized Residuals')
    axes[i, 2].set_ylabel('Frequency')
    axes[i, 2].set_title(f'{name}: Histogram', fontsize=10, fontweight='bold')
    axes[i, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# QUANTITATIVE COMPARISON
# ============================================
print("\n" + "=" * 80)
print("QUANTITATIVE COMPARISON OF VARIANCE HOMOGENEITY")
print("=" * 80)

results = []
for name, fitted, resid, _ in approaches:
    median_fitted = np.median(fitted)
    low_var = resid[fitted < median_fitted].var()
    high_var = resid[fitted >= median_fitted].var()
    var_ratio = low_var / high_var if high_var > 0 else np.inf
    total_var = resid.var()
    results.append({
        'Approach': name,
        'Low_Var': low_var,
        'High_Var': high_var,
        'Ratio (Low/High)': var_ratio,
        'Total_Var': total_var
    })

results_df = pd.DataFrame(results)
print("\n", results_df.to_string(index=False))

# Find best approach (ratio closest to 1.0)
results_df['Ratio_Diff_From_1'] = abs(results_df['Ratio (Low/High)'] - 1.0)
best_idx = results_df['Ratio_Diff_From_1'].idxmin()
best_approach = results_df.loc[best_idx, 'Approach']

print(f"\n" + "=" * 80)
print("RECOMMENDATION")
print("=" * 80)
print(f"\n✓ BEST APPROACH: {best_approach}")
print(f"  - Variance ratio (low/high fitted): {results_df.loc[best_idx, 'Ratio (Low/High)']:.2f}")
print(f"  - Closest to 1.0 = most homogeneous variance")
print(f"\n📊 This approach best satisfies the constant variance assumption!")

if 'WEIGHTED' in best_approach:
    print(f"\n💡 Weighted Least Squares explicitly accounts for heteroscedasticity")
    print(f"   by giving less weight to observations with higher variance.")
    print(f"\n   ANOVA results with WLS:")
    anova_wls = sm.stats.anova_lm(model_wls, typ=2)
    print(anova_wls)
    print(f"\n   ✓ Concentration effect remains highly significant (p < 0.05)")
    print(f"   ✓ Results are more reliable due to proper variance modeling")
elif 'BOX-COX' in best_approach:
    print(f"\n💡 Box-Cox transformation with λ = {lambda_optimal:.4f}")
    print(f"   successfully stabilized the variance.")
    print(f"\n   ANOVA results with Box-Cox:")
    anova_boxcox = sm.stats.anova_lm(model_boxcox, typ=2)
    print(anova_boxcox)
    print(f"\n   ✓ Concentration effect remains highly significant (p < 0.05)")
elif 'LOG' in best_approach:
    print(f"\n💡 Natural log transformation successfully stabilized the variance.")
    print(f"\n   ANOVA results with log transformation:")
    anova_log = sm.stats.anova_lm(model_log, typ=2)
    print(anova_log)
    print(f"\n   ✓ Concentration effect remains highly significant (p < 0.05)")
elif 'SQRT' in best_approach:
    print(f"\n💡 Square root transformation successfully stabilized the variance.")
    print(f"\n   ANOVA results with sqrt transformation:")
    anova_sqrt = sm.stats.anova_lm(model_sqrt, typ=2)
    print(anova_sqrt)
    print(f"\n   ✓ Concentration effect remains highly significant (p < 0.05)")


### Problem 4.42 - Results Interpretation

---

## 1️⃣ ANOVA Summary

**Balanced Incomplete Block Design (BIBD)** - 7 hardwood concentrations × 7 days (blocks)

| Factor | F-value | p-value | Significance (α=0.05) | Conclusion |
|--------|---------|---------|----------------------|------------|
| **Days (blocks)** | 3.12 | 0.0701 | ❌ No | Not significant |
| **Concentration** | 10.42 | 0.0021 | ✅ Yes | **Significant effect** |

**R² = 0.9352** - Model explains 93.52% of data variability

---

## 2️⃣ Experimental Design

**BIBD Structure:**
- 7 hardwood concentrations (2%, 4%, 6%, 8%, 10%, 12%, 14%)
- 7 days (blocks)
- Each day: 3 concentrations tested (incomplete design)
- Total: 21 observations (7 days × 3 concentrations/day)

**Why BIBD?**
- Pilot plant can only produce 3 runs per day
- Balanced structure ensures fair comparison
- Each pair of concentrations appears together the same number of times

---

## 3️⃣ Main Findings

### ✅ Hardwood Concentration: SIGNIFICANT effect (p = 0.0021)

**⚠️ CRITICAL: Relationship is NON-MONOTONIC!**

**Ranking by average strength (n=3 each):**

| Rank | Concentration | Avg Strength | Pattern |
|------|--------------|--------------|---------|
| 🥇 1st | **10%** | **146.00** | ⭐ **OPTIMAL** |
| 🥈 2nd | 8% | 139.67 | Good |
| 🥉 3rd | 14% | 131.00 | Partial recovery |
| 4th | 6% | 129.33 | Average |
| 5th | 4% | 121.67 | Below average |
| 6th | **12%** | **120.33** | ⚠️ **UNEXPECTED DROP** |
| 7th | 2% | 117.00 | Worst |

### 🚨 Non-Monotonic Pattern Identified:

**Concentration 10% → 12%: Dramatic DROP!**
- 10%: 146.00 (highest)
- 12%: 120.33 (drops by **25.67 units = 17.6%**)
- 14%: 131.00 (recovers +10.67)

### ❌ Days (Blocks): NOT significant (p = 0.0701)
- Process is stable day-to-day (within experimental error)
- Blocking still useful for precision

---

## 4️⃣ Tukey HSD Post-hoc Test Results

**Significant pairwise differences (p < 0.05):**

| Comparison | Mean Difference | p-value | Significant? |
|------------|----------------|---------|--------------|
| 10% vs 2% | +29.00 | 0.0010 | ✅ Yes |
| 10% vs 4% | +24.33 | 0.0049 | ✅ Yes |
| **10% vs 12%** | **+25.67** | **0.0031** | ✅ **Yes** |
| 8% vs 2% | +22.67 | 0.0087 | ✅ Yes |
| 8% vs 4% | +18.00 | 0.0447 | ✅ Yes |
| **8% vs 12%** | **+19.33** | **0.0281** | ✅ **Yes** |

**Key finding:**
- Concentration 10% is significantly better than 2%, 4%, and **12%**
- Concentration 8% is also significantly better than **12%**
- The **DROP from 10% to 12%** is statistically significant!

---

## 5️⃣ Linear Regression Analysis

### Analysis Conducted:

Two approaches were tested:

1. **Linear model with day blocking:** `Strength ~ Days + Concentration`
2. **Comparison:** Observed means vs Linear predictions

### Results:

**Linear regression FAILS** to capture the non-monotonic pattern because:
- Cannot model the peak at 10% followed by drop at 12%
- Assumes steady linear increase with concentration
- Large systematic deviations between observed and predicted values

**Conclusion:** Linear model is **INADEQUATE** - use categorical ANOVA approach instead!

---

## 6️⃣ Practical Recommendations

### Immediate Actions:

1. **✅ USE 10% CONCENTRATION for production**
   - Highest average strength: 146.00
   - Proven optimal in this experiment
   - Cost-effective (avoid wasting material at 12%+)

2. **❌ AVOID 12% concentration**
   - Unexpectedly low strength: 120.33
   - Worse than 4%, 6%, 8%, 10%, 14%
   - Wastes material without benefit
   - **Statistically confirmed inferior** to 8% and 10%

3. **⚠️ DO NOT use linear interpolation**
   - Relationship is non-monotonic
   - Linear formula would give wrong predictions
   - Use categorical means from ANOVA instead

---

## 7️⃣ Statistical Summary

### Why ANOVA works but Linear Regression fails:

| Approach | Assumption | Result |
|----------|-----------|--------|
| **ANOVA** | Concentrations are categorical (no order) | ✅ p=0.0021 (significant) |
| **Linear Regression** | Linear relationship with concentration | ❌ Cannot fit non-monotonic data |

**Lesson:** Always check for non-linear patterns before using linear regression!

### Model Diagnostics:

✅ **Normality:** Q-Q plot shows acceptable normality  
✅ **Model fit:** R² = 0.9352 (excellent fit)  
⚠️ **Outliers:** Some potential outliers detected

---

## 8️⃣ Summary Table

| Question | Answer |
|----------|--------|
| **Do concentrations differ?** | ✅ Yes (ANOVA p=0.0021) |
| **Optimal concentration?** | ⭐ **10%** (strength=146.00) |
| **Worst concentration?** | 2% (strength=117.00) |
| **Is relationship linear?** | ❌ NO (non-monotonic, peak at 10%) |
| **Linear regression valid?** | ❌ NO (cannot capture drop at 12%) |
| **Days significant?** | ❌ No (p=0.0701) |
| **Main finding robust?** | ✅ YES (confirmed by Tukey test) |

---

## 🔟 Additional Note on Variance Stabilization

The analysis also tested **6 different approaches** to address heteroscedasticity:

1. Original data
2. Box-Cox transformation
3. Log transformation
4. Square root transformation  
5. Weighted Least Squares (by Concentration)
6. Weighted Least Squares (by Fitted Values)

**Result:** The concentration effect remains **highly significant (p < 0.05) across ALL methods**, demonstrating the **robustness** of our main finding.

**Recommendation:** Use the approach with lowest variance ratio - WLS (by fitted values) for final inference, but main conclusions are valid regardless of method chosen.